In [1]:
import numpy as np
from scipy.special import binom
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImagePath


bernstein = lambda n, k, t: binom(n,k)* t**k * (1.-t)**(n-k)

def bezier(points, num=200):
    N = len(points)
    t = np.linspace(0, 1, num=num)
    curve = np.zeros((num, 2))
    for i in range(N):
        curve += np.outer(bernstein(N - 1, i, t), points[i])
    return curve

class Segment():
    def __init__(self, p1, p2, angle1, angle2, **kw):
        self.p1 = p1; self.p2 = p2
        self.angle1 = angle1; self.angle2 = angle2
        self.numpoints = kw.get("numpoints", 100)
        r = kw.get("r", 0.3)
        d = np.sqrt(np.sum((self.p2-self.p1)**2))
        self.r = r*d
        self.p = np.zeros((4,2))
        self.p[0,:] = self.p1[:]
        self.p[3,:] = self.p2[:]
        self.calc_intermediate_points(self.r)

    def calc_intermediate_points(self,r):
        self.p[1,:] = self.p1 + np.array([self.r*np.cos(self.angle1),
                                    self.r*np.sin(self.angle1)])
        self.p[2,:] = self.p2 + np.array([self.r*np.cos(self.angle2+np.pi),
                                    self.r*np.sin(self.angle2+np.pi)])
        self.curve = bezier(self.p,self.numpoints)


def get_curve(points, **kw):
    segments = []
    for i in range(len(points)-1):
        seg = Segment(points[i,:2], points[i+1,:2], points[i,2],points[i+1,2],**kw)
        segments.append(seg)
    curve = np.concatenate([s.curve for s in segments])
    return segments, curve

def ccw_sort(p):
    d = p-np.mean(p,axis=0)
    s = np.arctan2(d[:,0], d[:,1])
    return p[np.argsort(s),:]

def get_bezier_curve(a, rad=0.2, edgy=0):
    """ given an array of points *a*, create a curve through
    those points. 
    *rad* is a number between 0 and 1 to steer the distance of
          control points.
    *edgy* is a parameter which controls how "edgy" the curve is,
           edgy=0 is smoothest."""
    p = np.arctan(edgy)/np.pi+.5
    a = ccw_sort(a)
    a = np.append(a, np.atleast_2d(a[0,:]), axis=0)
    d = np.diff(a, axis=0)
    ang = np.arctan2(d[:,1],d[:,0])
    f = lambda ang : (ang>=0)*ang + (ang<0)*(ang+2*np.pi)
    ang = f(ang)
    ang1 = ang
    ang2 = np.roll(ang,1)
    ang = p*ang1 + (1-p)*ang2 + (np.abs(ang2-ang1) > np.pi )*np.pi
    ang = np.append(ang, [ang[0]])
    a = np.append(a, np.atleast_2d(ang).T, axis=1)
    s, c = get_curve(a, r=rad, method="var")
    x,y = c.T
    return x,y, a


def get_random_points(n=5, scale=0.8, mindst=None, rec=0):
    """ create n random points in the unit square, which are *mindst*
    apart, then scale them."""
    mindst = mindst or .7/n
    a = np.random.rand(n,2)
    d = np.sqrt(np.sum(np.diff(ccw_sort(a), axis=0), axis=1)**2)
    if np.all(d >= mindst) or rec>=200:
        return a*scale
    else:
        return get_random_points(n=n, scale=scale, mindst=mindst, rec=rec+1)

In [2]:
import numpy as np
from scipy.special import binom
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImagePath

# ... (Include all the previously explained functions here) ...

def generate_filled_bezier_image(size=16, num_points=5, scale=0.8, rad=0.2, edgy=0):
    # Generate random points
    points = get_random_points(n=num_points, scale=scale)

    # Get the x and y coordinates of the Bezier curve
    x, y, _ = get_bezier_curve(points, rad=rad, edgy=edgy)

    # Create a blank image
    img = Image.new("RGB", (size, size), (0, 0, 0))
    draw = ImageDraw.Draw(img)

    # Scale and shift the curve coordinates
    x = (x * (size - 1)).astype(int)
    y = (y * (size - 1)).astype(int)

    # Create a polygon from the curve points
    path = ImagePath.Path(list(zip(x, y)))

    # Fill the polygon
    draw.polygon(list(path), fill=(255, 255, 255))

    return img


In [3]:
import os
import random
from PIL import ImageChops

def xor_images(img1, img2):
    img1_bw = img1.convert("1")
    img2_bw = img2.convert("1")
    return ImageChops.logical_xor(img1_bw, img2_bw)

def generate_images(num_images=1000, img_size=64):
    output_dir = 'generated_images'
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_images):
        # Generate one or two images
        img1 = generate_filled_bezier_image(size=img_size)
        if random.random() < 0.5:
            img2 = generate_filled_bezier_image(size=img_size)
            final_img = xor_images(img1, img2)
        else:
            final_img = img1

        # Save the generated image
        final_img.save(os.path.join(output_dir, f'image_{i:05d}.png'))

generate_images(num_images=1000, img_size=64)

In [4]:
!zip -r generated_images.zip generated_images

  adding: generated_images/ (stored 0%)
  adding: generated_images/image_00961.png (stored 0%)
  adding: generated_images/image_00617.png (stored 0%)
  adding: generated_images/image_00755.png (stored 0%)
  adding: generated_images/image_00642.png (stored 0%)
  adding: generated_images/image_00452.png (stored 0%)
  adding: generated_images/image_00135.png (stored 0%)
  adding: generated_images/image_00474.png (stored 0%)
  adding: generated_images/image_00616.png (stored 0%)
  adding: generated_images/image_00931.png (stored 0%)
  adding: generated_images/image_00533.png (deflated 0%)
  adding: generated_images/image_00292.png (stored 0%)
  adding: generated_images/image_00408.png (stored 0%)
  adding: generated_images/image_00278.png (deflated 1%)
  adding: generated_images/image_00250.png (stored 0%)
  adding: generated_images/image_00528.png (stored 0%)
  adding: generated_images/image_00622.png (stored 0%)
  adding: generated_images/image_00737.png (stored 0%)
  adding: generated_i